In [62]:
import pandas as pd
import numpy as np

In [63]:
df = pd.read_csv("ML_Processed_Data.csv")

In [64]:
display(HTML(df.head().to_html()))

,Unnamed: 0,Year,Respondent_ID,Agency_Code,Loan_Type,Property_Type,Loan_Purpose,Occupancy,Amount,Action_Type,State_Code,County_Code,Income,Purchaser_Type,Denial_Reason_1,Rate_Spread,Application_Date_Indicator,Property_Location,USPS_Code,GDP,Real State Growth %,Rate,PMT,IncRat,Unemployemnt,AmorTerm,BoCreditScor,LTV
0,0,2015,659855,2,1,1,1,2,113,1,2,20,61,7,,NA,0,Alaska,AK,53.4,0.6,3.850577,529.790534,0.104221,6.5,360,4,0.75
1,1,2015,2888798,2,1,1,1,1,351,1,2,20,125,3,,NA,0,Alaska,AK,53.4,0.6,3.850577,1645.632542,0.157981,6.5,360,4,0.75
2,2,2015,2888798,2,1,1,1,1,148,2,2,20,69,0,,NA,0,Alaska,AK,53.4,0.6,3.850577,693.884947,0.120676,6.5,360,4,0.75
3,3,2015,2888798,2,1,1,1,1,167,2,2,20,69,0,,NA,0,Alaska,AK,53.4,0.6,3.850577,782.964771,0.136168,6.5,360,4,0.75
4,4,2015,2888798,2,1,1,1,1,197,1,2,20,43,6,,NA,0,Alaska,AK,53.4,0.6,3.850577,923.617125,0.257754,6.5,360,4,0.75


In [65]:
df.shape

(137509, 28)

In [66]:
df = df[(df["Action_Type"] == 1) | (df["Action_Type"] == 2) | (df["Action_Type"] == 3)| (df["Action_Type"] == 6)|(df["Action_Type"] == 7)]


"""
App
1
2
6

De
3
7"""

'\nApp\n1\n2\n6\n\nDe\n3\n7'

In [67]:
interested = ['Agency_Code', 'Loan_Type','Property_Type', 'Loan_Purpose', 'Occupancy', 'Amount', 'Action_Type','State_Code', 'County_Code', 'Income', 'Purchaser_Type ','Application_Date_Indicator','Property_Location', 'USPS_Code', 'GDP', 'Real State Growth %', 'Rate','PMT', 'IncRat', 'Unemployemnt', 'AmorTerm', 'BoCreditScor', 'LTV']


In [68]:
data = df[interested]

In [69]:
data.columns

Index(['Agency_Code', 'Loan_Type', 'Property_Type', 'Loan_Purpose',
       'Occupancy', 'Amount', 'Action_Type', 'State_Code', 'County_Code',
       'Income', 'Purchaser_Type ', 'Application_Date_Indicator',
       'Property_Location', 'USPS_Code', 'GDP', 'Real State Growth %', 'Rate',
       'PMT', 'IncRat', 'Unemployemnt', 'AmorTerm', 'BoCreditScor', 'LTV'],
      dtype='object')

In [70]:
cat = ['Agency_Code', 'Loan_Type', 'Property_Type', 'Loan_Purpose','Occupancy','USPS_Code' , 'County_Code','BoCreditScor', 'LTV' ]






no_cat = ['Amount', 'Income', 'GDP', 'Real State Growth %', 'Rate', 'PMT', 'IncRat', 'Unemployemnt']



In [71]:
## One Hot Encoding!
pre = pd.get_dummies(data, columns=cat, drop_first=True)

In [72]:
pre.shape

(126829, 340)

In [73]:
display(HTML(pre.head(10).to_html()))

,Amount,Action_Type,State_Code,Income,Purchaser_Type,Application_Date_Indicator,Property_Location,GDP,Real State Growth %,Rate,PMT,IncRat,Unemployemnt,AmorTerm,Occupancy_2,Occupancy_3,USPS_Code_AL,USPS_Code_AR,USPS_Code_AZ,USPS_Code_CA,USPS_Code_CO,USPS_Code_CT,USPS_Code_DC,USPS_Code_DE,USPS_Code_FL,USPS_Code_GA,USPS_Code_HI,USPS_Code_IA,USPS_Code_ID,USPS_Code_IL,USPS_Code_IN,USPS_Code_KS,USPS_Code_KY,USPS_Code_LA,USPS_Code_MA,USPS_Code_MD,USPS_Code_ME,USPS_Code_MI,USPS_Code_MN,USPS_Code_MO,USPS_Code_MS,USPS_Code_MT,USPS_Code_NC,USPS_Code_ND,USPS_Code_NE,USPS_Code_NH,USPS_Code_NJ,USPS_Code_NM,USPS_Code_NV,USPS_Code_NY,USPS_Code_OH,USPS_Code_OK,USPS_Code_OR,USPS_Code_PA,USPS_Code_RI,USPS_Code_SC,USPS_Code_SD,USPS_Code_TN,USPS_Code_TX,USPS_Code_UT,USPS_Code_VA,USPS_Code_VT,USPS_Code_WA,USPS_Code_WI,USPS_Code_WV,USPS_Code_WY,County_Code_3,County_Code_5,County_Code_6,County_Code_7,County_Code_9,County_Code_11,County_Code_12,County_Code_13,County_Code_14,County_Code_15,County_Code_17,County_Code_19,County_Code_20,County_Code_21,County_Code_23,County_Code_25,County_Code_27,County_Code_28,County_Code_29,County_Code_31,County_Code_33,County_Code_35,County_Code_36,County_Code_37,County_Code_39,County_Code_41,County_Code_43,County_Code_45,County_Code_47,County_Code_49,County_Code_51,County_Code_53,County_Code_55,County_Code_57,County_Code_59,County_Code_61,County_Code_63,County_Code_65,County_Code_67,County_Code_69,County_Code_71,County_Code_73,County_Code_75,County_Code_77,County_Code_78,County_Code_79,County_Code_81,County_Code_83,County_Code_85,County_Code_86,County_Code_87,County_Code_89,County_Code_91,County_Code_93,County_Code_95,County_Code_97,County_Code_99,County_Code_101,County_Code_103,County_Code_105,County_Code_107,County_Code_109,County_Code_111,County_Code_113,County_Code_115,County_Code_117,County_Code_119,County_Code_121,County_Code_122,County_Code_123,County_Code_125,County_Code_127,County_Code_129,County_Code_131,County_Code_133,County_Code_135,County_Code_137,County_Code_139,County_Code_141,County_Code_143,County_Code_145,County_Code_147,County_Code_149,County_Code_151,County_Code_153,County_Code_155,County_Code_157,County_Code_159,County_Code_161,County_Code_163,County_Code_165,County_Code_167,County_Code_169,County_Code_171,County_Code_173,County_Code_175,County_Code_177,County_Code_179,County_Code_181,County_Code_183,County_Code_185,County_Code_186,County_Code_187,County_Code_189,County_Code_191,County_Code_193,County_Code_195,County_Code_197,County_Code_199,County_Code_201,County_Code_203,County_Code_205,County_Code_207,County_Code_209,County_Code_211,County_Code_213,County_Code_215,County_Code_217,County_Code_219,County_Code_221,County_Code_223,County_Code_225,County_Code_227,County_Code_229,County_Code_231,County_Code_233,County_Code_235,County_Code_237,County_Code_239,County_Code_241,County_Code_245,County_Code_247,County_Code_249,County_Code_251,County_Code_253,County_Code_255,County_Code_257,County_Code_259,County_Code_261,County_Code_263,County_Code_265,County_Code_267,County_Code_271,County_Code_273,County_Code_275,County_Code_277,County_Code_279,County_Code_281,County_Code_285,County_Code_287,County_Code_289,County_Code_291,County_Code_293,County_Code_295,County_Code_297,County_Code_299,County_Code_303,County_Code_305,County_Code_307,County_Code_309,County_Code_311,County_Code_313,County_Code_317,County_Code_321,County_Code_323,County_Code_325,County_Code_327,County_Code_329,County_Code_331,County_Code_337,County_Code_339,County_Code_341,County_Code_343,County_Code_347,County_Code_349,County_Code_351,County_Code_355,County_Code_357,County_Code_359,County_Code_361,County_Code_363,County_Code_365,County_Code_367,County_Code_369,County_Code_371,County_Code_373,County_Code_375,County_Code_377,County_Code_379,County_Code_381,County_Code_393,County_Code_395,County_Code_397,County_Code_399,County_Code_401,County_Code_403,County_Code_407,County_Code_409,County_Code_413,County_Code_415,County_Code_417,County_Code_

In [74]:
## Normalizaation
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [75]:
scale = pd.DataFrame(data = pre)
scale[no_cat] = scaler.fit_transform(pre[no_cat])

# Show an example of a record with scaling applied
display(scale.head())

,Amount,Action_Type,State_Code,Income,Purchaser_Type,Application_Date_Indicator,Property_Location,GDP,Real State Growth %,Rate,...,County_Code_750,County_Code_760,County_Code_770,County_Code_775,County_Code_790,County_Code_800,County_Code_810,County_Code_820,County_Code_830,County_Code_840
0,0.004667,1,2,0.006001,7,0,Alaska,0.009385,0.486842,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.014584,1,2,0.012402,3,0,Alaska,0.009385,0.486842,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.006125,2,2,0.006801,0,0,Alaska,0.009385,0.486842,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0.006917,2,2,0.006801,0,0,Alaska,0.009385,0.486842,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.008167,1,2,0.004201,6,0,Alaska,0.009385,0.486842,0.0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
scale.shape

(126829, 340)

In [77]:
scale.columns

Index(['Amount', 'Action_Type', 'State_Code', 'Income', 'Purchaser_Type ',
       'Application_Date_Indicator', 'Property_Location', 'GDP',
       'Real State Growth %', 'Rate',
       ...
       'County_Code_750', 'County_Code_760', 'County_Code_770',
       'County_Code_775', 'County_Code_790', 'County_Code_800',
       'County_Code_810', 'County_Code_820', 'County_Code_830',
       'County_Code_840'],
      dtype='object', length=340)

In [78]:
## Balancing the Dataframe with 7K approved and 7K declined
s1 = scale[scale["Action_Type"] == 1]
s2 = scale[scale["Action_Type"] == 2]
s6 = scale[scale["Action_Type"] == 6]
s3 = scale[scale["Action_Type"] == 3]
s7 = scale[scale["Action_Type"] == 7]

In [79]:
total = s1.shape[0] + s2.shape[0] + s6.shape[0]
p = 7000*100 / total 
p
s6.shape[0]

2872

In [80]:
a = s1.head(6000)
a= a.append(s2.head(500))
a= a.append(s6.head(500))

In [81]:
d = s3.head(6997)
d = d.append(s7.head(3))


In [82]:
# Appedning the dataframe
final = a.append(d)

#Shuffling the dataframe
df = final.sample(frac=1).reset_index(drop=True)

In [83]:
y = []
a = 0
d = 0
for i in final["Action_Type"]:
    if i ==1 or i ==2 or i ==6:
        a +=1
        y.append(1)
        
    elif i ==3 or i==7:
        d +=1
        y.append(0)

In [84]:
del final["Action_Type"]

In [85]:
del final['Property_Location']

In [86]:
final.columns

Index(['Amount', 'State_Code', 'Income', 'Purchaser_Type ',
       'Application_Date_Indicator', 'GDP', 'Real State Growth %', 'Rate',
       'PMT', 'IncRat',
       ...
       'County_Code_750', 'County_Code_760', 'County_Code_770',
       'County_Code_775', 'County_Code_790', 'County_Code_800',
       'County_Code_810', 'County_Code_820', 'County_Code_830',
       'County_Code_840'],
      dtype='object', length=338)

In [87]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final, 
                                                    y, 
                                                    test_size = 0.3, 
                                                    random_state = 0)


print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))
type(X_train)

Training set has 9800 samples.
Testing set has 4200 samples.


pandas.core.frame.DataFrame

In [88]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [89]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [90]:
pred = clf.predict(X_test)

In [91]:
pred

array([1, 0, 0, ..., 1, 0, 1])

In [92]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, pred))




[[2005  133]
 [  14 2048]]


In [93]:
from sklearn.metrics import accuracy_score

In [94]:
accuracy_score(y_test, pred)

0.965

## Accuracy: 96.5